In [ ]:
!pip3 install ultralytics
!pip install torch torchvision torchaudio

In [ ]:
import os, zipfile, requests, yaml, shutil, random
from pathlib import Path
from tqdm import tqdm
import torch
from ultralytics import YOLO
from pathlib import Path
import yaml
import random as rn
import shutil
import os
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Enlazamos las imagenes con los labels

folder = "/content/drive/MyDrive/dataset_YOLO/images"
folder_labels = "/content/drive/MyDrive/dataset_YOLO/labels"

image_path = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith((".jpeg"))]
label_path = [os.path.join(folder_labels, f) for f in os.listdir(folder_labels) if f.endswith((".txt"))]
dataset = []

for path_i in image_path:
  index_i= re.findall(r'\d+', path_i)

  for path_l in label_path:
    index_l= re.findall(r'\d+', path_l)
    if index_i == index_l:
      dataset.append([path_i, path_l])
img_data =[]
label_data = []

for data in dataset:

  img_data.append(data[0])
  label_data.append(data[1])



Creamos el conjuntos de entranamiento y el de validación

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(img_data, label_data, test_size=0.2, random_state=42)


In [ ]:
#Separacion en carpetas de nuestros datos de entrenamiento y validacion

X_train = [ Path(dir)  for dir in X_train]
y_train = [ Path(dir)  for dir in y_train]
X_val = [ Path(dir)  for dir in X_val]
y_val = [ Path(dir)  for dir in y_val]


destino_X_train = Path("/content/drive/MyDrive/dataset_YOLO/dataset/images/train")
destino_y_train = Path("/content/drive/MyDrive/dataset_YOLO/dataset/labels/train")
destino_X_val = Path("/content/drive/MyDrive/dataset_YOLO/dataset/images/val")
destino_y_val = Path("/content/drive/MyDrive/dataset_YOLO/dataset/labels/val")

destinos = [destino_X_train, destino_y_train, destino_X_val, destino_y_val]


for destino in destinos:
  if destino.exists():
      shutil.rmtree(destino)
  destino.mkdir(parents=True)


for (origen_x, origen_y) in zip(X_train, y_train):

    shutil.copy(origen_x, destino_X_train)
    shutil.copy(origen_y, destino_y_train)


for (origen_x, origen_y) in zip(X_val, y_val):

    shutil.copy(origen_x, destino_X_val)
    shutil.copy(origen_y, destino_y_val)



In [ ]:
#Variables globales para el entrenamiento y analisis de precision

DATA_DIR = "/content/drive/MyDrive/dataset_YOLO/dataset"
YAML_OUT = "/content/drive/MyDrive/dataset_YOLO/dataset/custom.yaml"
MODEL_WEIGHTS = "yolov8n.pt"
EPOCHS = 300
IMG_SIZE = 640
FREEZE_LAYERS = 10

In [ ]:
#Funcion para crear archivo yaml

def create_yaml(data_root: Path, yaml_path: Path):
    cfg = {
        "path": str(data_root),
        "train": "images/train",
        "val": "images/val",
        "test": "",
        "nc": 1,
        "names": {
            0: "silla_ocupada"
        }
    }
    with open(yaml_path, "w") as f:
        yaml.safe_dump(cfg, f)
    print(f"YAML creado en {yaml_path}")

In [ ]:
#Funcion que realiza el Fine tuning de el modelo con los datos proporcionados

def fine_tune(yaml_path: Path, freeze_depth: int):
    print("Cargando modelo…")
    model = YOLO(MODEL_WEIGHTS)

    print(f"Iniciando fine‑tuning {EPOCHS} épocas, freeze={freeze_depth}")
    model.train(
        data=str(yaml_path),
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=-1,
        freeze=freeze_depth,
        device='gpu',
        lr0=0.0005,
        patience=300,
        verbose=True,
    )
    return model

In [ ]:
#Funcion de clasificacion binaria de una imagen a partir de un modelo que ha pasado por un proceso de fine-tunning

def clasificacion(model: YOLO, img):
    res = model(img)
    boxes = res[0].boxes
    ocupado = any([model.names[int(cls)] == "silla_ocupada" for cls in boxes.cls])
    return "clasificacion:OCUPADO" if ocupado else "clasificacion:DESOCUPADO"

In [ ]:
#Funcion que analiza el modelo con los datos de validacion

def validate_and_infer(model: YOLO, yaml_path: Path):
    print("Validación…")
    metrics = model.val(data=str(yaml_path))
    print(metrics)


    val_imgs = list((Path(DATA_DIR) / "images" / "val").glob("*.jpeg"))

    for img in val_imgs:
        print(f"Inferencia en {img.name}")
        res = model(img)
        res[0].show()
        print(clasificacion(model, img))


In [ ]:
torch.manual_seed(0)

In [ ]:
create_yaml(DATA_DIR, YAML_OUT)


YAML creado en /content/drive/MyDrive/dataset_YOLO/dataset/custom.yaml


In [ ]:
#Entrenamiendo del modelo
#Por favor volver a entrenar el modelo, esto debido a que ya sobrepasamos las limitaciones de GPU que nos proporciona Colab

yolo_model = fine_tune(YAML_OUT, FREEZE_LAYERS)

Cargando modelo…
Iniciando fine‑tuning 300 épocas, freeze=10
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/dataset_YOLO/dataset/custom.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto

100%|██████████| 755k/755k [00:00<00:00, 13.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 

train: Scanning /content/drive/MyDrive/dataset_YOLO/dataset/labels/train.cache... 35 images, 18 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     19/300         0G      1.239      2.274       1.29          3        640: 100%|██████████| 3/3 [00:20<00:00,  6.71s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.43s/it]

                   all          9         10          1      0.684      0.808      0.536

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



  0%|          | 0/3 [00:08<?, ?it/s]


KeyboardInterrupt: 

El modelo ha sido entrenado con learning rates=0.005,0.01 y epocas=50,100,200,300. Los mejores resultaron se obtuvieron en cantidades de epocas mas grandes y learning rate mas pequenos. las mediciones de box_loss y cls_loos fueron menores que el 0.4. lamentablementa por la poca cantidad de datos es posible que no haya una precision muy alta

In [ ]:
validate_and_infer(yolo_model, YAML_OUT)

In [ ]:
# modelo para segmentar imagenes
model_seg = YOLO("yolov8n-seg.pt")

100%|██████████| 6.74M/6.74M [00:00<00:00, 75.0MB/s]


In [ ]:
#Segmentacion de imagenes obtenidas

val_imgs = list((Path(DATA_DIR) / "images" / "val").glob("*.jpeg"))

for img in val_imgs:
  res = yolo_model(img)
  res_seg=model_seg(res[0])
  res_seg[0].show()